# fct_monthly_routes

In [12]:
import datetime
import sys

import _operator_grain_route_dir_visuals as _report_operator_visuals
import _sql_query
import google.auth
import pandas as pd
from google.cloud import bigquery
from IPython.display import HTML, Image, Markdown, display, display_html
from loguru import logger
from omegaconf import OmegaConf
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SEGMENT_GCS
from calitp_data_analysis import geography_utils

readable_dict = OmegaConf.load("readable2.yml")

import altair as alt
import geopandas as gpd
import shapely
from typing import Literal, Union

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_name = "City and County of San Francisco"

In [4]:
df = _sql_query.download_with_pandas_gbq(
    project="cal-itp-data-infra-staging",
    filename="tiffany_mart_gtfs_rollup.fct_monthly_routes",
)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_monthly_routes`
        WHERE month_first_day >=  DATE('2025-01-01')
    


/opt/conda/lib/python3.11/site-packages/pandas_gbq/gbq.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa


Downloading: 100%|██████████|
download time: 0:02:26.438747


In [9]:
df.columns

Index(['name', 'year', 'month', 'month_first_day', 'route_name',
       'direction_id', 'shape_id', 'shape_array_key', 'n_trips', 'pt_array'],
      dtype='object')

In [7]:
def convert_to_gdf(
    df: pd.DataFrame, 
    geom_col: str,
    geom_type: Literal["point", "line"]
) -> gpd.GeoDataFrame:
    """
    For stops, we want to make pt_geom a point.
    For vp_path and shapes, we want to make pt_array a linestring.
    """
    if geom_type == "point":
        df["geometry"] = [shapely.wkt.loads(x) for x in df[geom_col]]

    elif geom_type == "line":
        df["geometry"] = df[geom_col].apply(geography_utils.make_linestring)

    gdf = gpd.GeoDataFrame(
        df.drop(columns = geom_col), geometry="geometry", 
        crs="EPSG:4326"
    )

    return gdf

In [13]:
point_df = df.pipe(
        convert_to_gdf,
        "pt_array",
        "point"
    )

TypeError: Input must be valid geometry objects: [<POINT (-122.275 38.249)> <POINT (-122.275 38.249)>
 <POINT (-122.275 38.249)> <POINT (-122.275 38.249)>
 <POINT (-122.275 38.249)> <POINT (-122.275 38.249)>
 <POINT (-122.275 38.249)> <POINT (-122.275 38.249)>
 <POINT (-122.274 38.249)> <POINT (-122.274 38.249)>
 <POINT (-122.274 38.25)> <POINT (-122.274 38.25)>
 <POINT (-122.274 38.25)> <POINT (-122.274 38.25)>
 <POINT (-122.275 38.25)> <POINT (-122.275 38.25)>
 <POINT (-122.275 38.251)> <POINT (-122.275 38.251)>
 <POINT (-122.275 38.251)> <POINT (-122.275 38.251)>
 <POINT (-122.275 38.251)> <POINT (-122.275 38.251)>
 <POINT (-122.275 38.252)> <POINT (-122.275 38.252)>
 <POINT (-122.275 38.252)> <POINT (-122.275 38.252)>
 <POINT (-122.275 38.252)> <POINT (-122.275 38.252)>
 <POINT (-122.275 38.253)> <POINT (-122.275 38.253)>
 <POINT (-122.275 38.253)> <POINT (-122.275 38.253)>
 <POINT (-122.275 38.253)> <POINT (-122.275 38.253)>
 <POINT (-122.275 38.254)> <POINT (-122.275 38.254)>
 <P

In [ ]:
df.columns

In [ ]:
df = df.pipe(portfolio_utils.standardize_portfolio_organization_names)

In [ ]:
sf_only = df.loc[df.analysis_name == analysis_name]

In [ ]:
sf_only = sf_only.loc[sf_only.month_first_day == '2025-10-01T00:00:00.000000000']

In [ ]:
type(sf_only)

In [ ]:
sf_only.explore()

In [ ]:
df.route_name.unique()

In [ ]:
df.drop(columns = ["geometry", "pt_array"]).sample(5)

In [ ]:
route_typologies_df = _sql_query.download_with_pandas_gbq(
    project="cal-itp-data-infra-staging",
    filename="tiffany_mart_gtfs_rollup.fct_monthly_schedule_rt_route_direction_summary",
)

In [ ]:
route_typologies_df.sample()

In [ ]:
route_typologies_df[["route_name"]].drop_duplicates().sort_values(by = ["route_name"])

In [ ]:
df[["route_name"]].drop_duplicates().sort_values(by = ["route_name"])

In [ ]:
route_typologies_df2 = route_typologies_df[
    [
        "name",
        "month_first_day",
        "route_name",
        "route_type",
        "route_color",
        "route_typology",
    ]
]

In [ ]:
m1 = pd.merge(
    df,
    route_typologies_df2,
    on=[
        "name",
        "month_first_day",
        "route_name",
    ],
    how="outer",
    indicator=True,
)

In [ ]:
m1._merge.value_counts()